### 前置作業

1. 安裝好相關tquant環境
2. 設定好.env
3. 放好 strategy.py

In [32]:
# load .env
import os

import tejapi
from dotenv import load_dotenv

load_dotenv()
tejapi.ApiConfig.api_key = os.environ.get("TEJAPI_KEY")
tejapi.ApiConfig.api_base = os.environ.get("TEJAPI_BASE")

In [33]:
# 載入套件

import importlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import strategy
import TejToolAPI
from strategy import PercentageIndicatorStrategy
from zipline import run_algorithm
from zipline.algo.pipeline_algo import *
from zipline.data import bundles
from zipline.data.data_portal import get_bundle
from zipline.pipeline import CustomFactor, Pipeline
from zipline.pipeline.data import TWEquityPricing
from zipline.pipeline.data.dataset import Column, DataSet
from zipline.pipeline.domain import TW_EQUITIES
from zipline.pipeline.factors import DailyReturns
from zipline.pipeline.loaders.frame import DataFrameLoader
from zipline.sources.TEJ_Api_Data import get_universe
from zipline.utils.calendar_utils import get_calendar

In [34]:
# 載入外部資料

start = '2016-01-01'
end = '2024-05-24'

start_dt = pd.Timestamp(start, tz='utc')
end_dt = pd.Timestamp(end, tz='utc')

tickers = get_universe(start, end, mkt_bd_e=['TSE', 'OTC'], stktp_c=['普通股'])

columns = [
    'd0003',
    'd0004',
    'd0007',
    'eps',
    'Outstanding_Shares_1000_Shares',
]

external_data = TejToolAPI.get_history_data(ticker=tickers,
                                            columns=columns,
                                            start=start,
                                            end=end)
external_data['mdate'] = pd.to_datetime(external_data['mdate'], utc=True)

[2024-05-26 15:12:15.935286]: INFO: get_universe_TW: Filters：{'mkt_bd_e': ['TSE', 'OTC'], 'stktp_c': ['普通股']}


In [22]:
# 下載回測(價量)資料

os.environ['mdate'] = start + ' ' + end
os.environ['ticker'] = " ".join(tickers + ['IR0001'])

In [ ]:
# 下載回測(價量)資料
!zipline ingest -b tquant

In [61]:
bundle_name = 'tquant'
bundle = bundles.load(bundle_name)

df_bundle = get_bundle(bundle_name=bundle_name,
                       calendar_name='TEJ',
                       start_dt=start_dt,
                       end_dt=end_dt)
df_bundle

In [62]:
class CustomDataset(DataSet):
    Basic_Earnings_Per_Share_TTM = Column(dtype=float)
    Basic_Earnings_Per_Share_Q = Column(dtype=float)
    Outstanding_Shares_1000_Shares = Column(dtype=float)
    MoM_Monthly_Sales = Column(dtype=float)
    YoY_Monthly_Sales = Column(dtype=float)
    domain = TW_EQUITIES


sids = bundle.asset_finder.equities_sids
assets = bundle.asset_finder.retrieve_all(sids)
symbol_mapping_sid = {i.symbol: i.sid for i in assets}
transform_data = external_data.set_index(['coid', 'mdate']).unstack('coid')
transform_data = transform_data.rename(columns=symbol_mapping_sid)
transform_data

In [37]:
inputs = [
    CustomDataset.Basic_Earnings_Per_Share_TTM,
    CustomDataset.Basic_Earnings_Per_Share_Q,
    CustomDataset.Outstanding_Shares_1000_Shares,
    CustomDataset.MoM_Monthly_Sales, CustomDataset.YoY_Monthly_Sales
]
custom_loader = {
    i:
    DataFrameLoader(column=i,
                    baseline=transform_data.xs(i.name, level=0, axis=1))
    for i in inputs
}

In [40]:
class EPSRatio(CustomFactor):
    inputs = [CustomDataset.Basic_Earnings_Per_Share_TTM]
    window_length = 756

    def compute(self, today, assets, out, eps):
        out[:] = eps[-1] / eps[0]


class GrowthFactorX(CustomFactor):
    inputs = [EPSRatio()]
    window_length = 1

    def compute(self, today, assets, out, eps_ratio):
        out[:] = np.cbrt(eps_ratio) - 1


class EPSYOY(CustomFactor):
    inputs = [CustomDataset.Basic_Earnings_Per_Share_Q]
    window_length = 252

    def compute(self, today, assets, out, eps):
        out[:] = (eps[-1] - eps[0]) / eps[0]


class ValueStrategy(strategy.PercentageIndicatorStrategy):

    def compute_signals(self):
        outstanding_shares = CustomDataset.Outstanding_Shares_1000_Shares.latest
        yoy_monthly_sales = CustomDataset.YoY_Monthly_Sales.latest
        growth_factor_x = GrowthFactorX()
        eps_yoy = EPSYOY()
        # condition 1
        growth_factor_x_greater_than_0_25 = growth_factor_x > 0.25
        # condition 2
        growth_yoy_greater_than_0_25 = eps_yoy > 0.25
        # condition 3
        yoy_monthly_sales_greater_than_0_1 = yoy_monthly_sales > 10

        filter = growth_factor_x_greater_than_0_25 & growth_yoy_greater_than_0_25 & yoy_monthly_sales_greater_than_0_1
        signal = outstanding_shares.zscore(mask=filter)
        return Pipeline(columns={
            'longs': signal.bottom(30),
        })


In [63]:

def initialize(context):
    context.strategy = ValueStrategy(assets, start_dt, end_dt)
    context.strategy.initialize(context)


def before_trading_start(context, data):
    context.strategy.before_trading_start(context, data)


def analyze(context, perf):

    fig = plt.figure(figsize=(16, 12))

    # First chart(累計報酬)
    ax = fig.add_subplot(311)
    ax.set_title('Strategy Results')
    ax.plot(perf['algorithm_period_return'],
            linestyle='-',
            label='algorithm period return',
            linewidth=3.0)
    ax.plot(perf['benchmark_period_return'],
            linestyle='-',
            label='benchmark period return',
            linewidth=3.0)
    ax.legend()
    ax.grid(False)
    # Second chart(ending_cash)->觀察是否超買
    ax = fig.add_subplot(312)
    ax.plot(perf['ending_cash'],
            label='ending_cash',
            linestyle='-',
            linewidth=1.0)
    ax.axhline(y=1, c='r', linewidth=0.3)
    ax.legend()
    ax.grid(True)


result = run_algorithm(pd.Timestamp('2020-04-01', tz='utc'),
                       end=end_dt,
                       initialize=initialize,
                       before_trading_start=before_trading_start,
                       capital_base=1e6,
                       data_frequency='daily',
                       bundle_timestamp=pd.Timestamp('2016-01-01', tz='utc'),
                       bundle='tquant',
                       analyze=analyze,
                       trading_calendar=get_calendar("TEJ"),
                       custom_loader=custom_loader)

In [64]:
import pyfolio
from pyfolio.utils import extract_rets_pos_txn_from_zipline

returns, positions, transactions = extract_rets_pos_txn_from_zipline(result)
benchmark_rets = result.benchmark_return
pyfolio.tears.create_full_tear_sheet(returns=returns,
                                     positions=positions,
                                     transactions=transactions,
                                     benchmark_rets=benchmark_rets
                                    )